# Ford GoBike System Data
## by Robert Grütze


## Table of Contents

1. Intro
2. Data Wrangling
3. 


## 1. Intro

This data set includes information about individual rides made in a bike-sharing system covering the greater San Francisco Bay area.

> Each trip is anonymized and includes:

- Trip Duration (seconds)
- Start Time and Date
- End Time and Date
- Start Station ID
- Start Station Name
- Start Station Latitude
- Start Station Longitude
- End Station ID
- End Station Name
- End Station Latitude
- End Station Longitude
- Bike ID
- User Type (Subscriber or Customer – “Subscriber” = Member or “Customer” = Casual)

**Following questions will be adressed in this notebook:**
> *When are most trips taken in terms of time of day, day of the week, or month of the year?*

> *How long does the average trip take?*

> *Does the above depend on if a user is a subscriber or customer?*

> *Which bike IDs were used in particular and which not?*

> *Are there any special patterns regading the renting behaviour to be observed?*


## 2. Wrangling

In [122]:
# import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import requests, zipfile
import re
import os 
import calendar
import math

%matplotlib inline

### Gather Data

In [90]:
# gather 2017 data

# unzip files in order to handle filesize limitations in git
with zipfile.ZipFile("data/2017-fordgobike-tripdata.csv.zip") as z:
    # load dataset from data subir
    with z.open("2017-fordgobike-tripdata.csv") as f:
        df_2017 = pd.read_csv(f)

In [78]:
# gather 2019 data

# gather img predictions by downloading image-predictions.tsv  to /data
url_ford = 'https://s3.amazonaws.com/baywheels-data/2019{:02d}-fordgobike-tripdata.csv.zip'
url_baywheels = 'https://s3.amazonaws.com/baywheels-data/2019{:02d}-baywheels-tripdata.csv.zip'

df_list = list()
switch_idx = 4


# loop through each monthly file
for idx in range(12):
    
    # replace url string
    if idx < switch_idx:
        url_tem = url_ford.format(idx+1)
    else:
        url_tem = url_baywheels.format(idx+1)
        
    # send request
    response = requests.get(url_tem)
    with open(url_tem.split('/')[-1], mode = 'wb') as outfile:
        
        # load data
        source = url_tem.split('/')[-1]
        destination = os.path.join('data', source)
        outfile.write(response.content)
        os.rename(source, destination)
        
        # extract data
        with zipfile.ZipFile(destination) as z:
            with z.open(source[:-4]) as f:
                df_tem = pd.read_csv(f)
                if idx < switch_idx:
                    df_list.append(df_tem)
                else:
                    df_list.append(df_tem)


In [87]:
# concat monthly dataframe to yearly one
df_2019 = []
df_2019 = pd.concat(df_list, axis=0, sort=False)
df_2019.head()

duration_sec                start_time                  end_time  \
0         80825  2019-01-31 17:57:44.6130  2019-02-01 16:24:49.8640   
1         65900  2019-01-31 20:58:33.8860  2019-02-01 15:16:54.1730   
2         62633  2019-01-31 18:06:52.9240  2019-02-01 11:30:46.5300   
3         44680  2019-01-31 19:46:09.7190  2019-02-01 08:10:50.3180   
4         60709  2019-01-31 14:19:01.5410  2019-02-01 07:10:51.0650   

   start_station_id           start_station_name  start_station_latitude  \
0             229.0    Foothill Blvd at 42nd Ave               37.775745   
1               4.0  Cyril Magnin St at Ellis St               37.785881   
2             245.0       Downtown Berkeley BART               37.870139   
3              85.0      Church St at Duboce Ave               37.770083   
4              16.0      Steuart St at Market St               37.794130   

   start_station_longitude  end_station_id              end_station_name  \
0              -122.213037           196.0       Grand Ave at Perkins St   
1              -122.408915           134.0        Valencia St at 24th St   
2              -122.268422           157.0          65th St at Hollis St   
3              -122.429156            53.0        Grove St at Divisadero   
4              -122.394430            28.0  The Embarcadero at Bryant St   

   end_station_latitude  end_station_longitude  bike_id   user_type  \
0             37.808894            -122.256460     4861  Subscriber   
1             37.752428            -122.420628     5506  Subscriber   
2             37.846784            -122.291376     2717    Customer   
3             37.775946            -122.437777     4557    Customer   
4             37.787168            -122.388098     2100    Customer   

  bike_share_for_all_trip rental_access_method  
0                      No                  NaN  
1                      No                  NaN  
2                      No                  NaN  
3                      No                  NaN  
4                      No                  NaN

In [ ]:
# rename dataset
df = df_2019

### Assess Data

In [96]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2506983 entries, 0 to 150101
Data columns (total 15 columns):
duration_sec               int64
start_time                 object
end_time                   object
start_station_id           float64
start_station_name         object
start_station_latitude     float64
start_station_longitude    float64
end_station_id             float64
end_station_name           object
end_station_latitude       float64
end_station_longitude      float64
bike_id                    int64
user_type                  object
bike_share_for_all_trip    object
rental_access_method       object
dtypes: float64(6), int64(2), object(7)
memory usage: 306.0+ MB


In [101]:
df.shape

(2506983, 15)

In [102]:
df.head()

duration_sec                start_time                  end_time  \
0         80825  2019-01-31 17:57:44.6130  2019-02-01 16:24:49.8640   
1         65900  2019-01-31 20:58:33.8860  2019-02-01 15:16:54.1730   
2         62633  2019-01-31 18:06:52.9240  2019-02-01 11:30:46.5300   
3         44680  2019-01-31 19:46:09.7190  2019-02-01 08:10:50.3180   
4         60709  2019-01-31 14:19:01.5410  2019-02-01 07:10:51.0650   

   start_station_id           start_station_name  start_station_latitude  \
0             229.0    Foothill Blvd at 42nd Ave               37.775745   
1               4.0  Cyril Magnin St at Ellis St               37.785881   
2             245.0       Downtown Berkeley BART               37.870139   
3              85.0      Church St at Duboce Ave               37.770083   
4              16.0      Steuart St at Market St               37.794130   

   start_station_longitude  end_station_id              end_station_name  \
0              -122.213037           196.0       Grand Ave at Perkins St   
1              -122.408915           134.0        Valencia St at 24th St   
2              -122.268422           157.0          65th St at Hollis St   
3              -122.429156            53.0        Grove St at Divisadero   
4              -122.394430            28.0  The Embarcadero at Bryant St   

   end_station_latitude  end_station_longitude  bike_id   user_type  \
0             37.808894            -122.256460     4861  Subscriber   
1             37.752428            -122.420628     5506  Subscriber   
2             37.846784            -122.291376     2717    Customer   
3             37.775946            -122.437777     4557    Customer   
4             37.787168            -122.388098     2100    Customer   

  bike_share_for_all_trip rental_access_method  
0                      No                  NaN  
1                      No                  NaN  
2                      No                  NaN  
3                      No                  NaN  
4                      No                  NaN

In [100]:
df.describe()

duration_sec  start_station_id  start_station_latitude  \
count  2.506983e+06      2.426249e+06            2.506983e+06   
mean   8.076483e+02      1.465047e+02            3.776506e+01   
std    1.974714e+03      1.223171e+02            1.363698e-01   
min    6.000000e+01      3.000000e+00            0.000000e+00   
25%    3.590000e+02      4.700000e+01            3.776931e+01   
50%    5.710000e+02      1.050000e+02            3.778053e+01   
75%    8.870000e+02      2.430000e+02            3.779539e+01   
max    9.121100e+05      4.980000e+02            4.551000e+01   

       start_station_longitude  end_station_id  end_station_latitude  \
count             2.506983e+06    2.424081e+06          2.506983e+06   
mean             -1.223499e+02    1.427044e+02          3.776422e+01   
std               3.089648e-01    1.214296e+02          2.392885e-01   
min              -1.225143e+02    3.000000e+00          0.000000e+00   
25%              -1.224130e+02    4.300000e+01          3.777003e+01   
50%              -1.223983e+02    1.010000e+02          3.778076e+01   
75%              -1.222914e+02    2.390000e+02          3.779587e+01   
max               0.000000e+00    4.980000e+02          4.551000e+01   

       end_station_longitude       bike_id  
count           2.506983e+06  2.506983e+06  
mean           -1.223459e+02  2.789833e+04  
std             7.080417e-01  1.146067e+05  
min            -1.225143e+02  4.000000e+00  
25%            -1.224117e+02  1.952000e+03  
50%            -1.223981e+02  4.420000e+03  
75%            -1.222934e+02  9.682000e+03  
max             0.000000e+00  9.999410e+05

In [98]:
df.duplicated().sum()

0

In [105]:
df.isnull().sum()


duration_sec               0
start_time                 0
end_time                   0
start_station_id           0
start_station_name         0
start_station_latitude     0
start_station_longitude    0
end_station_id             0
end_station_name           0
end_station_latitude       0
end_station_longitude      0
bike_id                    0
user_type                  0
dtype: int64

In [113]:
df.dtypes

duration_sec                        int64
start_time                 datetime64[ns]
end_time                   datetime64[ns]
start_station_id                   object
start_station_name                 object
start_station_latitude            float64
start_station_longitude           float64
end_station_id                     object
end_station_name                   object
end_station_latitude              float64
end_station_longitude             float64
bike_id                             int64
user_type                          object
dtype: object

**observed quality and titiness issues with dataset**
- format of *start_time* and *end_time* is not datetime
- any id should be stored as a string
- drop rows with null values 
- drop unused columns bike_share_for_all_trip, rental_access_method

**wanted features to be obtained from the dataset**
- distance of individual rides
- extract *day*, *day of the week* and *month of the year*


### Clean Data 

**Define**:
    - drop unused columns
    - drop rows with null values

In [104]:
# columns
df.drop(['rental_access_method', 'bike_share_for_all_trip'], axis=1, inplace=True)

# nulls
df.dropna(inplace=True)


**Define**:
    - fix column format

In [112]:
# fix IDs as string
df.start_station_id = df.start_station_id.astype(str)
df.end_station_id   = df.end_station_id.astype(str)
df.bike_id          = df.bike_id.astype(str)


# fix time columns
df['start_time'] = pd.to_datetime(df['start_time'].dt.date)
df['end_time']   = pd.to_datetime(df['end_time'].dt.date)


**Define**:
    - extract Monute/Hour/Day/Weekday/Month/Year in seperate columns

In [117]:
# hour
df['start_time_hour'] = df['start_time'].dt.hour
df['end_time_hour']   = df['end_time'].dt.hour

In [116]:
# week
df['start_time_weekday'] = df['start_time'].dt.weekday_name
df['end_time_weekday']   = df['end_time'].dt.weekday_name

df['start_time_weekday'] = df['start_time'].dt.weekday.apply(lambda x: calendar.day_abbr[x])
df['end_time_weekday']   = df['end_time'].dt.weekday.apply(lambda x: calendar.day_abbr[x])

In [ ]:
# month
df['start_time_month'] = df['start_time'].dt.month.astype(int)
df['end_time_month']   = df['end_time'].dt.month.astype(int)

In [ ]:
# year
df['start_time_year']  = df['start_time'].dt.year.astype(int)
df['end_time_year']    = df['end_time'].dt.year.astype(int)


In [118]:
df.head()

duration_sec start_time   end_time start_station_id  \
0         80825 2019-01-31 2019-02-01            229.0   
1         65900 2019-01-31 2019-02-01              4.0   
2         62633 2019-01-31 2019-02-01            245.0   
3         44680 2019-01-31 2019-02-01             85.0   
4         60709 2019-01-31 2019-02-01             16.0   

            start_station_name  start_station_latitude  \
0    Foothill Blvd at 42nd Ave               37.775745   
1  Cyril Magnin St at Ellis St               37.785881   
2       Downtown Berkeley BART               37.870139   
3      Church St at Duboce Ave               37.770083   
4      Steuart St at Market St               37.794130   

   start_station_longitude end_station_id              end_station_name  \
0              -122.213037          196.0       Grand Ave at Perkins St   
1              -122.408915          134.0        Valencia St at 24th St   
2              -122.268422          157.0          65th St at Hollis St   
3              -122.429156           53.0        Grove St at Divisadero   
4              -122.394430           28.0  The Embarcadero at Bryant St   

   end_station_latitude  end_station_longitude  bike_id   user_type  \
0             37.808894            -122.256460     4861  Subscriber   
1             37.752428            -122.420628     5506  Subscriber   
2             37.846784            -122.291376     2717    Customer   
3             37.775946            -122.437777     4557    Customer   
4             37.787168            -122.388098     2100    Customer   

  start_time_weekday end_time_weekday  start_time_hour  end_time_hour  
0                Thu              Fri                0              0  
1                Thu              Fri                0              0  
2                Thu              Fri                0              0  
3                Thu              Fri                0              0  
4                Thu              Fri                0              0

In [119]:
df.shape

(2407259, 17)

**Define**:
- get distance between start and endpoint

*using proposed solution from:* 
https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude

In [127]:
def distance(origin, destination):
    """
    Calculate the Haversine distance.

    Parameters
    ----------
    origin : tuple of float
        (lat, long)
    destination : tuple of float
        (lat, long)

    Returns
    -------
    distance_in_km : float

    Examples
    --------
    >>> origin = (48.1372, 11.5756)  # Munich
    >>> destination = (52.5186, 13.4083)  # Berlin
    >>> round(distance(origin, destination), 1)
    504.2
    """
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371  # km

    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = (math.sin(dlat / 2) * math.sin(dlat / 2) +
         math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) *
         math.sin(dlon / 2) * math.sin(dlon / 2))
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = radius * c

    return d

In [128]:
df['distance_km'] = df.apply(lambda x: distance((x['start_station_latitude'], x['start_station_longitude']), (x['end_station_latitude'], x['end_station_longitude'])), axis=1)
#df['distance'] = df['distance']*0.621371



### What is the structure of your dataset?

> Zipped, csv files from the year 2019 with 2407259 entries for 17 columns. No null values and no duplicates were found.

### What is/are the main feature of interest in your dataset?

- renting duration since this has a linear correlation to lyfts revenue

### What features in the dataset do you think will help support your investigation into your feature(s) of interest?

- start_station_latitude and start_station_longitude
- end_station_latitude and end_station_longitude
- user_type
- start_time and end_time

## Univariate Exploration

### Question 1:
*When are most trips taken in terms of time of day, day of the week, or month of the year?*


> Make sure that, after every plot or related series of plots, that you
include a Markdown cell with comments about what you observed, and what
you plan on investigating next.

### Discuss the distribution(s) of your variable(s) of interest. Were there any unusual points? Did you need to perform any transformations?

> Your answer here!

### Of the features you investigated, were there any unusual distributions? Did you perform any operations on the data to tidy, adjust, or change the form of the data? If so, why did you do this?

> Your answer here!

## Bivariate Exploration

> In this section, investigate relationships between pairs of variables in your
data. Make sure the variables that you cover here have been introduced in some
fashion in the previous section (univariate exploration).

### Talk about some of the relationships you observed in this part of the investigation. How did the feature(s) of interest vary with other features in the dataset?

> Your answer here!

### Did you observe any interesting relationships between the other features (not the main feature(s) of interest)?

> Your answer here!

## Multivariate Exploration

> Create plots of three or more variables to investigate your data even
further. Make sure that your investigations are justified, and follow from
your work in the previous sections.

### Talk about some of the relationships you observed in this part of the investigation. Were there features that strengthened each other in terms of looking at your feature(s) of interest?

> Your answer here!

### Were there any interesting or surprising interactions between features?

> Your answer here!

> At the end of your report, make sure that you export the notebook as an
html file from the `File > Download as... > HTML` menu. Make sure you keep
track of where the exported file goes, so you can put it in the same folder
as this notebook for project submission. Also, make sure you remove all of
the quote-formatted guide notes like this one before you finish your report!